In [10]:
"""
collect region data in the latest 10 years from china national data. (for 31 provinces)
I am trying to refactor this scripts's spider method, 
focus on: creat a pool to store target urls, and handle request parallelly.
# pool: learn from scrapy design architechure
# gevent / yeild for url parallel handling.
what's done:
    #get datas into dataframe type
""" 


"\ncollect region data in the latest 10 years from china national data. (for 31 provinces)\nI am trying to refactor this scripts's spider method, \nfocus on: creat a pool to store target urls, and handle request parallelly.\n# pool: learn from scrapy design architechure\n# gevent / yeild for url parallel handling.\nwhat's done:\n    #get datas into dataframe type\n"

In [11]:
import urllib,urllib2,json
import httplib
addr="http://data.stats.gov.cn"
ndurl = "http://data.stats.gov.cn/english/easyquery.htm"

In [12]:
from pandas import DataFrame, Series
import pandas as pd
%matplotlib inline
from pandas import DataFrame, Series
import seaborn as sns

In [13]:
def check_with_valuecode(valuecode):
    wdcode="zb"  #
    #valuecode="A0101"  #data collection A0501**  LAST12
    dfwds="[{%22wdcode%22%3A%22"+ wdcode +"%22%2C%22valuecode%22%3A%22"+ valuecode + "%22}]"
    data={
        "colcode":"sj",  #more than one year data
        "dbcode":"hgnd", #hgyd-month hgjd-quater
        "m":"QueryData",
        "rowcode":"zb",
        }
    test_url = ndurl+'?'+urllib.urlencode(data)+"&dfwds=%s"%(dfwds)+"&wds=[]"
    #print test_url
    resp = urllib2.urlopen(test_url)
    result = json.loads(resp.read(),strict=False)
    #print result
    #how to parse the data frame
    if result["returncode"] != 200 : 
        print "return code error for:",valuecode
        return False
    codelist=[]
    for _value in result["returndata"]["wdnodes"][0]["nodes"]:
        codelist.append(_value['code'])
    return codelist

In [14]:
def get_reg_df_with_valuecode(valuecode):
    print "INFO:",valuecode
    wdcode="reg"  #
    wds="[{%22wdcode%22%3A%22"+ "zb" +"%22%2C%22valuecode%22%3A%22"+ valuecode + "%22}]"
    dfwds="[]"
    data={
        "colcode":"sj",  #more than one year data
        "dbcode":"fsnd", #hgyd-month hgjd-quater
        "m":"QueryData",
        "rowcode":"reg",
        }
    test_url = ndurl+'?'+urllib.urlencode(data)+"&dfwds=%s"%(dfwds) +"&wds=%s"%wds
    #print test_url
    #return 
    resp = urllib2.urlopen(test_url)
    result = json.loads(resp.read(),strict=False)
    #print result
    #how to parse the data frame
    if result["returncode"] != 200 : 
        print "return code error for:",valuecode
        return []
    
    if not result["returndata"]["datanodes"]: 
        print "Warning: skip data code ",valuecode
        return DataFrame()
    #dataname:
    dataname=result["returndata"]["wdnodes"][0]["nodes"][0]["cname"]
    #get citycode mapping dataframe
    citycode_list=[]
    for _index in result["returndata"]["wdnodes"][1]["nodes"]:
        citycode ={
            "citycode": _index["code"],
            "cityname": _index["cname"],
        }
        citycode_list.append(citycode)
    citycode_df=DataFrame(citycode_list)
    #get data and city mapping
    data_list=[]
    for _data in result["returndata"]["datanodes"]:
        data_value = {
            "citycode":_data["code"].split(".")[2].split("_")[0],
            "data":_data["data"]["data"],
            "year":_data["code"].split(".")[3],
            "datacode":_data["code"].split(".")[1].split("_")[0],
            "dataname":dataname,
        }
        data_list.append(data_value)
    data_df=DataFrame(data_list)    
    
    #merge data
    df=pd.merge(data_df,citycode_df)
    return df


In [15]:
testdf=get_reg_df_with_valuecode("A050401")

INFO: A050401


In [16]:
testdf

,citycode,data,datacode,dataname,year,cityname
0,110000,2102.6465,A050401,Total Investment in Residential Buildings in t...,2014,Beijing
1,110000,2026.7532,A050401,Total Investment in Residential Buildings in t...,2013,Beijing
2,110000,1872.9647,A050401,Total Investment in Residential Buildings in t...,2012,Beijing
3,110000,1942.6368,A050401,Total Investment in Residential Buildings in t...,2011,Beijing
4,110000,1662.2000,A050401,Total Investment in Residential Buildings in t...,2010,Beijing
5,110000,1034.6785,A050401,Total Investment in Residential Buildings in t...,2009,Beijing
6,110000,1028.2178,A050401,Total Investment in Residential Buildings in t...,2008,Beijing
7,110000,1091.3903,A050401,Total Investment in Residential Buildings in t...,2007,Beijing
8,110000,951.3017,A050401,Total Investment in Residential Buildings in t...,2006,Beijing
9,110000,877.8951,A050401,Total Investment in Residential Buildings in t...,2005,Beijing


In [17]:
testdf.to_csv(path_or_buf="./reg_10year.csv",encoding="utf-8")